In [1]:
import pandas as pd
import polars as pl
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from collections import defaultdict
import warnings
from tqdm import tqdm
import pickle

tqdm.pandas()
warnings.filterwarnings("ignore")

In [42]:
year = 22
df = f"./data/CampaignFin20/indivs{year}.txt"
donors_csv = f"./data/CampaignFin{year}/donors{year}.csv"
names = f"./data/USIN.csv"

In [43]:
donors = pd.read_csv(donors_csv)
donors["lastname"] = donors["name"].apply(lambda x: str(x).split(",")[0].lower())
donors.head(10)

,contrib_id,name,name_new,orgname,ultorg,gender,occupation,employer,total_donated,donation_count,avg_donation,med_donation,lastname
0,,"HOME, 30119 SETTLE",30119 settle home,[24T Contribution],NaN,,NOT EMPLOYED,NOT EMPLOYED,1.649385e+09,115116,1.432803e+04,25.0,home
1,U00000003641,"SOROS, GEORGE",george soros,Soros Fund Management,NaN,M,EXECUTIVE,SOROS FUND MANAGEMENT,1.800173e+08,204,8.824376e+05,2900.0,soros
2,U00000036901,"UIHLEIN, RICHARD",richard uihlein,Uline Inc,NaN,M,CEO/OWNER,ULINE,8.422153e+07,369,2.282426e+05,2900.0,uihlein
3,U00000036551,"GRIFFIN, KENNETH C",kenneth c griffin,Citadel LLC,NaN,M,CEO,CITADEL LLC,7.437550e+07,317,2.346230e+05,2900.0,griffin
4,U00000042451,"YASS, JEFF",jeff yass,Susquehanna International Group,NaN,M,MANAGING DIRECTOR,SIG,5.632450e+07,49,1.149480e+06,5800.0,yass
5,U0000004705,"BANKMAN-FRIED, SAMUEL",samuel bankman-fried,FTX.US,NaN,M,CEO,FTX US,4.204488e+07,338,1.243931e+05,5000.0,bankman-fried
6,U00000046841,"MELLON, TIMOTHY",timothy mellon,Investor,NaN,M,INVESTOR,SELF-EMPLOYED,4.174640e+07,31,1.346658e+06,2900.0,mellon
7,U00000037041,"BLOOMBERG, MICHAEL R",michael r bloomberg,Bloomberg Lp,NaN,M,FOUNDER,BLOOMBERG INC.,4.133051e+07,39,1.059757e+06,200000.0,bloomberg
8,U0000003235,"EYCHANER, FRED",fred eychaner,Newsweb Corp,NaN,M,OWNER,NEWSWEB CORP,3.782750e+07,234,1.616560e+05,10000.0,eychaner
9,U00000003611,"SCHWARZMAN, STEPHEN",stephen schwarzman,Blackstone Group,NaN,M,CHAIRMAN CEO,BLACKSTONE,3.754490e+07,281,1.336117e+05,2900.0,schwarzman


In [5]:
# https://github.com/philipperemy/name-dataset
df_indian = pd.read_csv("./data/IN.csv")
df_indian.columns = ['firstname', 'lastname', 'gender', 'ethnicity']
df_indian['firstname'] = df_indian['firstname'].apply(lambda x: x.split(" ")[0] if " " in str(x) else str(x))
df_indian['lastname'] = df_indian['lastname'].apply(lambda x: x.split(" ")[-1] if " " in str(x) else str(x))
df_indian['name'] = df_indian['firstname'].apply(lambda x: x.lower()) + ' ' + df_indian['lastname'].apply(lambda x: x.lower())
df_indian["ethnic"] = df_indian["ethnicity"].apply(lambda x: "ind")
df_indian = df_indian[['lastname', 'name', 'ethnic']]
df_indian.head(10)

,lastname,name,ethnic
0,Kumar,kapil kumar,ind
1,Aggarwal,mitali aggarwal,ind
2,Jangra,vikas jangra,ind
3,Lungay,ravi lungay,ind
4,Yadav,jagat yadav,ind
5,Kumari,kajal kumari,ind
6,Kumar,vimal kumar,ind
7,Singham,surya singham,ind
8,Hanjra,kawaljarnail hanjra,ind
9,Rathor,kuldeep rathor,ind


In [6]:
sample = pd.read_csv("./manual/top_donors_all.csv")
sample["firstname"] = sample["name"].apply(lambda x: x.split(",")[-1] if "," in x else str(x))
sample["lastname"] = sample["name"].apply(lambda x: x.split(",")[0] if "," in x else str(x))
sample["name_new"] = sample["firstname"].apply(lambda x: x.lower()) + ' ' + sample["lastname"].apply(lambda x: x.lower())
sample["ethnic"] = sample["actual"].apply(lambda x: "ind" if x == True else "not")
sample = sample[['name', 'lastname', 'name_new', 'actual', 'ethnic']]
sample.head(10)

,name,lastname,name_new,actual,ethnic
0,"for america, amy",for america,amy for america,NaN,not
1,"bloomberg, michael",bloomberg,michael bloomberg,False,not
2,"steyer, thomas f",steyer,thomas f steyer,False,not
3,"mellon, timothy",mellon,timothy mellon,False,not
4,"adelson, miriam o dr",adelson,miriam o dr adelson,False,not
5,"adelson, sheldon g mr",adelson,sheldon g mr adelson,False,not
6,"uihlein, richard e",uihlein,richard e uihlein,False,not
7,"griffin, kenneth",griffin,kenneth griffin,False,not
8,"schwarzman, stephen a",schwarzman,stephen a schwarzman,False,not
9,"jurvetson, karla",jurvetson,karla jurvetson,False,not


In [44]:
# get the most common indian last names
indian_lastnames = set(df_indian["lastname"].str.lower().head(5000))

# add the known indian-american last names
known_indian_lastnames = sample[sample["ethnic"] == "ind"]["lastname"].str.lower().tolist()
indian_lastnames.update(known_indian_lastnames)

# remove the names that are more commonly american
indian_lastnames = [x for x in indian_lastnames if x not in ["king", "mann", "ray", "gill", "m", "paul", "farmer", "john", "kay", 
                                                             "r", "abraham", "camp", "g", "p", "power", "song", "k", "kang", "binder", 
                                                             "joy", "null", "sell", "n", "ko", "shi", "ricker", "robin", "rambo", "ku", 
                                                             "back", "banker", "hans", "sandy", "grand", "ji", "bains", "bale", "duane", 
                                                             "don", "mi", "mall", "baden", "panter", "sing", "barra", "davi", 
                                                             "lasseter", "munger", "kaler", "maddy", "lucky", "pop", "shalom", "ro",
                                                             "rod", "sha", "pon", "bander", "barman", "biber", "goldy", "bobby", "bou", 
                                                             "sky", "nan", "done", "boy", "summy", "boro", "all", "rings", "vali",
                                                             "shing", "dema", "farm", "lather", "suny", "kalson", "payment", "si",
                                                             "ch", "sah", "come", "roy", "adam", "harry", "virk", "mian", "san",
                                                             "vik", "butte"]]

In [45]:
donors["ethnic"] = np.where(donors["lastname"].str.lower().isin(indian_lastnames), "ind", "not")
donors.sort_values(by="total_donated", ascending=False).loc[:9999, "ethnic"] = sample["ethnic"]
donors.to_csv(f"./output/donors{year}_pred_lastname.csv", index=False)
donors["ethnic"].value_counts()

ethnic
not    2743702
ind      17316
Name: count, dtype: int64

In [46]:
donors[donors["ethnic"] == "ind"]["lastname"].value_counts()[:10]

lastname
patel     2538
singh      983
shah       720
khan       706
gupta      384
ahmed      368
ali        345
kumar      321
reddy      304
sharma     285
Name: count, dtype: int64

In [47]:
donors[donors["lastname"] == "roy"]

,contrib_id,name,name_new,orgname,ultorg,gender,occupation,employer,total_donated,donation_count,avg_donation,med_donation,lastname,ethnic
46600,s00051139011,"ROY, ALEXANDER",alexander roy,Retired,NaN,M,RETIRED,RETIRED,20300.0,5,4060.000000,2900.0,roy,not
50343,a0000939990,"ROY, JAMES",james roy,NaN,NaN,M,LAWYER,DOMENGEAUX WRIGHT ROY & EDWARDS LLC,19085.0,153,124.738562,83.0,roy,not
59857,r0016819951,"ROY, ASIT",asit roy,[24T Contribution],NaN,M,NOT EMPLOYED,NOT EMPLOYED,16464.0,534,30.831461,15.0,roy,not
62072,i3004199421,"ROY, JOSEPHINE",josephine roy,National Republican Congressional Cmte,NaN,F,TEACHER,WEST NEW YORK BOARD OF ED,15853.0,267,59.374532,50.0,roy,not
69626,s0005113901@,"ROY, JENNIFER L",jennifer l roy,Retired,NaN,F,RETIRED,RETIRED,14500.0,4,3625.000000,2900.0,roy,not
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725453,r0002263211,"ROY, MICHAEL",michael roy,Vaughn D Thibodeau,NaN,M,TRUCK DRIVER,VAUGHN D THIBODEAU,11.0,1,11.000000,11.0,roy,not
2743083,r0014335626,"ROY, PARAMITA",paramita roy,Dentist,NaN,F,DENTIST,SELF-EMPLOYED,7.0,1,7.000000,7.0,roy,not
2753299,s0013690941,"ROY, RARITA",rarita roy,Unemployed,NaN,M,NOT EMPLOYED,NONE,5.0,1,5.000000,5.0,roy,not
2753477,s0007031065,"ROY, ROBERT",robert roy,Retired,NaN,M,RETIRED,RETIRED,5.0,1,5.000000,5.0,roy,not


In [48]:
donors[donors["ethnic"] == "ind"].sort_values(by="total_donated", ascending=False)

,contrib_id,name,name_new,orgname,ultorg,gender,occupation,employer,total_donated,donation_count,avg_donation,med_donation,lastname,ethnic
37,U0000004761,"SINGH, NISHAD",nishad singh,FTX.US,NaN,M,DIRECTOR OF ENGINEERING,FTX,8476000.0,148,57270.270270,2900.0,singh,ind
123,U00000040581,"ANWAR, S, JAVAID",javaid anwar,Midland Energy,NaN,M,PRESIDENT/CEO,MIDLAND ENERGY INC.,3232392.0,287,11262.689895,2900.0,anwar,ind
197,U00000033011,"KHOSLA, VINOD",vinod khosla,Khosla Ventures,NaN,M,VENTURE CAPITALIST,"KHOSLA VENTURES, LLC",2084900.0,30,69496.666667,5000.0,khosla,ind
260,U00000045811,"MANOCHERIAN, JED",jed manocherian,[24T Contribution],NaN,M,REAL ESTATE,SELF,1557000.0,236,6597.457627,2900.0,manocherian,ind
279,U00000047321,"KIANI, JOE",joe kiani,Masimo Corp,NaN,M,CEO,MASIMO,1460800.0,52,28092.307692,2900.0,kiani,ind
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2760077,h3001740863,"RAO, C K",c k rao,Medical Director,NaN,U,MEDICAL DIRECTOR,SELF-EMPLOYED,1.0,1,1.000000,1.0,rao,ind
2760147,r0023603976,"ALI, CASSAMO",cassamo ali,[24T Contribution],NaN,U,NaN,NaN,1.0,1,1.000000,1.0,ali,ind
2760397,r0017386326@,"BHATT, NACHIKETA MR",nachiketa mr bhatt,Nurse,...,M,SELF-EMPLOYED,SELF-EMPLOYED,1.0,1,1.000000,1.0,bhatt,ind
2760659,s0013681943,"ISLAM, SHAHISLAM",shahislam islam,[24T Contribution],NaN,U,NaN,NaN,1.0,1,1.000000,1.0,islam,ind
